In [3]:
!pip install pandas numpy scikit-learn
!pip install gradio

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Load datasets
true_df = pd.read_csv("True.csv")
fake_df = pd.read_csv("fake.csv")

# Add labels
true_df["label"] = "REAL"
fake_df["label"] = "FAKE"

# Combine and shuffle
df = pd.concat([true_df, fake_df], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)

# Combine title and text as input
df["content"] = df["title"] + " " + df["text"]

# Feature and label
X = df["content"]
y = df["label"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Model training
model = PassiveAggressiveClassifier(max_iter=50)
model.fit(X_train_tfidf, y_train)

# Prediction and evaluation
y_pred = model.predict(X_test_tfidf)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

# Results
print(f"Accuracy: {round(acc * 100, 2)}%")
print("Confusion Matrix:\n", cm)

# Test with sample
sample = ["The election results were rigged. It's all fake!"]
sample_tfidf = vectorizer.transform(sample)
print("Sample Prediction:", model.predict(sample_tfidf)[0])

import gradio as gr

def predict_news(text):
    transformed = vectorizer.transform([text])
    prediction = model.predict(transformed)[0]
    return prediction

interface = gr.Interface(fn=predict_news, inputs="text", outputs="text", title="Fake News Detector")
interface.launch()



Accuracy: 99.42%
Confusion Matrix:
 [[5868   33]
 [  32 5292]]
Sample Prediction: FAKE
* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
